In [ ]:
import gammapy
gammapy.__version__

In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import pickle

import setup
from utilities import Log, MyLabel
from grb import GammaRayBurst
from pop_plot import col_size, sig_legend

from astropy.visualization import quantity_support

# Transform warnings into errors - useful to find who is guilty !
import warnings
#warnings.filterwarnings('error')
warnings.filterwarnings('ignore')

os.environ['GAMMAPY_DATA'] =r'../../../input/gammapy-extra-master/datasets'


# A look to the input population files

## Get GRB data

In [ ]:
# Define your folder here
folder = "../../../input/lightcurves/LONG_FITS"
file_list = os.listdir(folder)
# file_list

# Create the population file

In [ ]:
# GRB population
nmax    = 2000

# Output folder 
outfolder   = "D:\CTA\Analyse\output"

# Prompt list
out_prompt  = open(Path(outfolder,"prompt_list.txt"),"w") # If defined
out_prompt=None

# Population file
out_pop = "grbpop_dominguez.bin" # populatin generic name for input/output

# Actions
store   = "read" #"read" # If "read" get a previously saved population file using "write"

debug = False # Debugging"


In [ ]:
# Reading a previously saved population
if store == "read":
    with open(Path(outfolder, out_pop), 'rb') as file:
        grbpop = pickle.load(file)
        print(" GRB read from disk :",len(grbpop))

if store != "read" or out_prompt: # Write population to disk and /or create visible prompt list
    grbpop  = []
    ipop    = 0
    iprompt = 0

    # Loop over (sub)-population from disk
    for igrb, file in enumerate(file_list[:nmax]):

        if ((nmax <= 10) or (np.mod(igrb,40) == 0)) and debug==False:
            print("#",igrb," ",end="")

        # Read file in directory"
        if Path(file).suffix == ".fits":
            grb =  GammaRayBurst.from_fits(Path(folder,file), ebl= "dominguez", newis=False, magnify=1)

            if debug:
                print(grb)
                print(grb.vis["North"].print(log=Log()))
                print(grb.vis["South"].print(log=Log()))

            if store == "write": 
                grbpop.append(grb)
                ipop+=1

            if (out_prompt):
                if grb.vis["North"].vis_prompt or grb.vis["South"].vis_prompt:
                    print("{:10s}".format(grb.name),file=out_prompt)
                    iprompt+=1

    print()           
    if out_prompt: 
        print("GRb with prompt visible : ",iprompt," -> List dumped into prompt_list.txt")
        out_prompt.close()    

    if store == "write":
        grbpop = np.array(grbpop)
        print(" GRB in array : ",len(grbpop))
        filename = Path(outfolder,out_pop)
        outfile  = open(filename,"wb")
        pickle.dump(grbpop,outfile)
        outfile.close()
        print(" GRB saved to : {}".format(filename))

print("All done !")

# Explore GRB properties

## Galactic plane coverage

In [ ]:
import astropy.units as u
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes,inset_axes
from astropy.coordinates import SkyCoord

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(20,7))
latmax = 2*u.deg
with quantity_support():
    dec = np.array([g.radec.dec.wrap_at(180*u.deg).value for g in grbpop])*u.degree
    ra  = np.array([g.radec.ra.wrap_at(180*u.deg).value for g in grbpop])*u.degree
#     dec = dec.wrap_at(180*u.deg)
#     ra  = ra.wrap_at(180*u.deg)
    c = SkyCoord(ra=ra, dec=dec, frame='icrs')
    lat = c.galactic.b
    ax1.hist(lat, bins=1000,label=MyLabel(lat))
    ax1.set_xlabel("Longitude (deg)")
    ax1.legend()
    
    mask = abs(lat.value)<=latmax.value
    lat_in = lat[mask]
    namelist = [g.name for g in grbpop[mask]]
    ax2.set_xlim([-1.05*latmax,1.05*latmax])
    ax2.axvline( latmax.value,color="red",ls="--",label="Lat. limit")
    ax2.axvline(-latmax.value,color="red",ls="--",label="Lat. limit")
    ax2.plot(lat_in,np.ones(len(lat_in)),ls="",marker="o",label=MyLabel(lat_in))
    ax2.set_xlabel("Latitude")

    i=0
    for l, txt in zip(lat_in, namelist):
        ax2.text(l,1.01+(i%5)*0.015,s=txt[5:],rotation=90)
        ax2.set_ylim(ymax=1.1)
        i+=1
    patches = sig_legend(ax2)
    ax2.legend(handles =patches)

In [ ]:
z     = np.array([grb.z for grb in grbpop])
Eiso  = np.array([grb.Eiso.value for grb in grbpop])*grbpop[0].Eiso.unit
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,5))
with quantity_support():
    ax1.hist(z)
    ax2.hist(Eiso)

In [ ]:
fig = plt.figure(figsize=(18,6))
ax = fig.add_subplot(111, projection="mollweide")

sc = ax.scatter(ra.to("rad"), dec.to("rad"),marker='o',c=z,s=20+5*z**2,cmap='rainbow',alpha=0.5)
ax.set_title("GRB z distribution")
cbar = fig.colorbar(sc,  orientation='vertical')
cbar.set_label('z', rotation=270)
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,6))
ax = fig.add_subplot(111, projection="aitoff")
ax.set_title("GRB Eiso distribution")

sc = ax.scatter(ra.to("rad"), dec.to("rad"),marker='o',c=np.log10(Eiso.value),
                s=20+10*(np.log10(Eiso.value)-50)**2,
                alpha=0.5,
                cmap='viridis')
cbar = fig.colorbar(sc,  orientation='vertical')
cbar.set_label('Log10(Eiso)', rotation=270)
plt.show()

## Number of E and t bins

In [ ]:
nEval = []
ntval = []
for g in grbpop:
    nEval.append(len(g.Eval))
    ntval.append(len(g.tval))
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax1.hist(nEval,label=MyLabel(nEval))
ax1.set_xlabel("Energy array length")
ax1.legend()
ax2.hist(ntval,label=MyLabel(ntval))
ax2.set_xlabel("Time array length")
ax2.legend()
plt.show()


# Analyze input spectra

In [ ]:
def get_max(grb,Eref=100*u.GeV,ax=None,plot=False,**kwargs):
    
    idxref  = np.abs(grb.Eval-Eref).argmin() # Get flux close to Eref
    fluxref = grb.fluxval[:,idxref] # Obtain time spectrum for that energy
    idmax   = fluxref.argmax() # Get time index of max value 

    if plot:
        if ax==None and plot: 
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,6))

        with quantity_support():
            ax.plot(np.log10(grb.tval.value),np.log10(fluxref.value),marker="o",**kwargs)
            # Get time of max
            fluxmax =  fluxref[idmax]
            tmax    = grb.tval[idmax]
            ax.axhline(y=fluxmax,ls="--")
            ax.axvline(x=np.log10(tmax.value),ls="--")
#             print(fluxmax)
    return idmax

In [ ]:
nmax=1000
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20,5))

for ax0, Eref in zip(ax,[50*u.GeV, 100*u.GeV, 200*u.GeV, 500*u.GeV]):
    print(Eref)
    tmax = []
    for grb in grbpop[:nmax]:
        idmax = get_max(grb,Eref=Eref, ax=ax,plot=False)
        tmax.append(grb.tval[idmax].value)
    tmax = np.array(tmax)*grb.tval[0].unit
    with quantity_support():
        ax0.hist(np.log10(tmax.value),bins=25,alpha=0.3,label=MyLabel(tmax,str(Eref),stat="med"))
    ax0.legend()

In [ ]:
grb = grbpop[0]
idx = (np.array([g(100*u.GeV).value for g in grb.spectra])).argmax()
print(idx)

In [ ]:
get_max(grb)